In [ ]:
aaa

In [1]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# see more columns and rows
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# read in previously made dataset
InjuryRecord = pd.read_csv('InjuryRecord.csv')
PlayList = pd.read_csv('PlayList_mod.csv')
CB_WR_plays=pd.read_csv('CB_WR_plays.csv')
ir_pl=pd.read_csv('ir_pl.csv')
CB_WR_plays.shape

cols=['PlayerKey','Position','PlayerGame','PlayerGamePlay','PlayKey','FieldType','PlayType','Weather_category','Temperature','Stadium']
mask = (ir_pl['Position']=='CB') | (ir_pl['Position']=='WR')
cb_wr_injured = ir_pl[mask][cols]
cb_wr_injured['Status'] = 'Injured'
# print(cb_wr_injured.shape)

cb_wr_inj_playkey = cb_wr_injured['PlayKey'].unique()
mask = (PlayList['Position']=='CB') | (PlayList['Position']=='WR') & (PlayList['Stadium'].notnull()) & (PlayList['Weather_category'].notnull())
cb_wr_all = PlayList[mask][cols]

# create mask that holds index of only healthy plays
mask = (cb_wr_all['Stadium'].notnull()) & (cb_wr_all['Weather_category'].notnull())
cb_wr_all = cb_wr_all[mask][cols]
non_inj_index=cb_wr_all.loc[~cb_wr_all['PlayKey'].isin(cb_wr_inj_playkey)].index

cb_wr_healthy=cb_wr_all.loc[non_inj_index].sample(n=185, random_state=111) # pick 185 random plays.  Injrrd plays make up 1/5 of the total plays used in this analysis
cb_wr_healthy['Status'] = 'Healthy'
# print(cb_wr_healthy.shape)

data=pd.concat([cb_wr_injured,cb_wr_healthy]).reset_index(drop=True)
data.shape


from matplotlib import gridspec
import matplotlib.patches as patches
legend_properties = {'weight':'bold','size':2}
scatter_kwargs = {'s':50,'cmap':'jet','alpha':0.7,'linewidth':.2,'edgecolor':'k'}
sns_scatter_kwargs = {'s':250,'alpha':1,'linewidth':2,'edgecolor':'k','marker':'d'}
font = {'color':  'black',
        'weight': 'bold',
        'size': 24}
fts=20
def sf(xy,title, xlabel, ylabel, font,fontticksize,rotate):
    xy.set_title(title,fontdict=font)
    xy.set_xlabel(xlabel,fontdict=font)
    xy.set_ylabel(ylabel,fontdict=font)
    
    for tick in xy.xaxis.get_ticklabels():
        tick.set(rotation=rotate) # set tick rotation
        tick.set_fontsize(fontticksize) # set tick font size
        tick.set_fontweight('bold') # set tick font weight
    for tick in xy.yaxis.get_ticklabels():
        tick.set_fontsize(fontticksize)
        tick.set(rotation=0)
        tick.set_fontweight('bold')
        
        

data_playkey = data['PlayKey'].unique()
gg=[]
for pt, play in enumerate(data_playkey[48:110]):
    mask = (CB_WR_plays['PlayKey']==play)

    cb_wr_play = CB_WR_plays[mask]
    metrics=[]
    metrics=[]
    for idx, step in enumerate(cb_wr_play['dir']):
        if cb_wr_play['dir'].iloc[idx+1]<90 and cb_wr_play['dir'].iloc[idx]>250:
            d_dir=cb_wr_play['dir'].iloc[idx+1]+(360-cb_wr_play['dir'].iloc[idx])

        elif cb_wr_play['dir'].iloc[idx+1]>250 and cb_wr_play['dir'].iloc[idx]<90:
            d_dir=-(cb_wr_play['dir'].iloc[idx]+(360-cb_wr_play['dir'].iloc[idx+1]))
        else:
            d_dir=cb_wr_play['dir'].iloc[idx+1]-cb_wr_play['dir'].iloc[idx]
            
            
        metrics.append((cb_wr_play['time'].iloc[idx+1],
                        cb_wr_play['event'].iloc[idx],
                        cb_wr_play['time'].iloc[idx+1]-cb_wr_play['time'].iloc[idx],                    
                        d_dir,
                        cb_wr_play['dis'].iloc[idx+1]-cb_wr_play['dis'].iloc[idx],
                        cb_wr_play['o'].iloc[idx+1]-cb_wr_play['o'].iloc[idx],
                        cb_wr_play['s'].iloc[idx+1]-cb_wr_play['s'].iloc[idx]))
        
        if idx==len(cb_wr_play['dir'])-2:
            break

            
    df=pd.DataFrame(metrics,columns=('time','event','delta_time','delta_direction','delta_distance','delta_orient','delta_speed'))

    fig=plt.figure()
    fig.set_size_inches(32,16)
    widths = [1,1]
    heights = [1,1,1]
    grid = fig.add_gridspec(ncols=2, nrows=3,wspace=0.05)#, width_ratios=widths, height_ratios=heights,left=0.05, right=0.48, wspace=0.05)
            
    ####################################################################################################################
    #############################             TIME                        ##############################################
    ####################################################################################################################
    ####################################################################################################################

    xy = fig.add_subplot(grid[0, 0])
    xy = plt.scatter(cb_wr_play['x'], cb_wr_play['y'],c=cb_wr_play['time'],**scatter_kwargs)
    cbar = plt.colorbar(xy)
    font_size = 18 # Adjust as appropriate.
    cbar.ax.tick_params(labelsize=font_size)
    cbar.ax.tick_params(color="red", width=5, length=10)
    cbar.ax.set_ylabel('Time', fontsize = 20, weight="bold")
    xy=sns.scatterplot(x='x',y='y',hue='event',data=cb_wr_play,legend=None,**sns_scatter_kwargs)
    sf(xy,'Time','x','y',font,fontticksize=fts,rotate=0)

    ####################################################################################################################
    #############################             SPEED                       ##############################################
    ####################################################################################################################
    ####################################################################################################################

    xy = fig.add_subplot(grid[0, 1])
    xy = plt.scatter(cb_wr_play["x"], cb_wr_play["y"],c=cb_wr_play['s'],**scatter_kwargs)
    cbar = plt.colorbar(xy)
    cbar.ax.tick_params(labelsize=font_size)
    cbar.ax.tick_params(color="red", width=5, length=10)
    cbar.ax.set_ylabel('Speed', fontsize = 20, weight="bold")
    xy=sns.scatterplot(x='x',y='y',hue='event',data=cb_wr_play,legend=None,**sns_scatter_kwargs)
    sf(xy,'Speed','x','y',font,fontticksize=fts,rotate=0)

    ####################################################################################################################
    #############################             ORIENTATION                 ##############################################
    ####################################################################################################################
    ####################################################################################################################

    xy = fig.add_subplot(grid[1, 0])
    xy = plt.scatter(cb_wr_play["x"], cb_wr_play["y"],c=cb_wr_play['o'],**scatter_kwargs)
    cbar = plt.colorbar(xy)
    cbar.ax.tick_params(color="red", width=5, length=10)
    cbar.ax.tick_params(labelsize=font_size)
    cbar.ax.set_ylabel('Orientation', fontsize = 20, weight="bold")
    xy=sns.scatterplot(x='x',y='y',hue='event',data=cb_wr_play,legend=None,**sns_scatter_kwargs)
    sf(xy,'Orientation','x','y',font,fontticksize=fts,rotate=0)

    # xy.set_ylim([0,53.3])
    # xy.set_xlim([0,120])

    ####################################################################################################################
    #############################             DIRECTION                   ##############################################
    ####################################################################################################################
    ####################################################################################################################

    xy = fig.add_subplot(grid[1, 1])
    xy = plt.scatter(cb_wr_play["x"], cb_wr_play["y"],c=cb_wr_play['dir'],**scatter_kwargs)
    cbar = plt.colorbar(xy)
    cbar.ax.tick_params(labelsize=font_size)
    cbar.ax.tick_params(color="red", width=5, length=10)
    cbar.ax.set_ylabel('Direction', fontsize = 20, weight="bold")
    xy=sns.scatterplot(x='x',y='y',hue='event',data=cb_wr_play,legend=None,**sns_scatter_kwargs)
    sf(xy,'Direction','x','y',font,fontticksize=fts,rotate=0)
#     xy.legend(bbox_to_anchor=(1.55, 0.5),prop=legend_properties)

    ####################################################################################################################
    #####################            CHANGE IN SPEED AND DIRECTION                ######################################
    ####################################################################################################################
    ####################################################################################################################
    xy = fig.add_subplot(grid[2, :])
    xy = plt.scatter(df["time"], df["delta_direction"],c=df['delta_speed'],s=150, cmap="seismic",alpha=.5,edgecolors='k')
    cbar = plt.colorbar(xy)
    cbar.ax.tick_params(color="red", width=5, length=10)
    cbar.ax.tick_params(labelsize=font_size)
    cbar.ax.set_ylabel('Change in Speed', fontsize = 20, weight="bold")
    xy=sns.scatterplot(x='time',y='delta_direction',hue='event',data=df,s=240,edgecolor='k',linewidth=2,marker='d')
    xy.set_xlim(0,df['time'].max()) # set y limits
    xy.set_xticks(np.arange(0,df['time'].max()+2,2)) # set number of y ticks
    sf(xy,'Change in Speed and Direction from Base Line','Time','Change in Direction',font,fontticksize=fts,rotate=0)
#     xy.legend(loc=2,prop=legend_properties)
#     xy.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    fig.subplots_adjust(bottom=0.0, hspace=0.3, wspace=0.10)
    plt.gcf().text(.92, .6, 'Play Info: \n'+
                   play + '\n'+
                   data.loc[data['PlayKey']==play]['PlayType'].iloc[0]+ '\n'+
                   data.loc[data['PlayKey']==play]['Status'].iloc[0]+'\n'+
                   data.loc[data['PlayKey']==play]['FieldType'].iloc[0]+ '\n'+
                   data.loc[data['PlayKey']==play]['Stadium'].iloc[0]+ '\n'+
                   data.loc[data['PlayKey']==play]['Weather_category'].iloc[0]+ '\n'+
                   str(data.loc[data['PlayKey']==play]['Temperature'].iloc[0]),
                   fontsize=29)

    # add a title that is centered over the figure
    sup_title= fig.suptitle('PlayKey '+play+': '+data.loc[data['PlayKey']==play]['PlayType'].iloc[0],fontsize=30,fontweight='bold')
    # adjst the space between title and plot
    sup_title.set_y(0.94)
    # assign legend properties
    legend_properties = {'weight':'bold','size':22}
# assign location of where the legend will be placed.  Change vaules to shift legend
    x_shift = 0.76
    y_shift = 0.5
    bbox = (fig.subplotpars.left+x_shift, fig.subplotpars.top-y_shift, fig.subplotpars.right-fig.subplotpars.left,.1)
#     assign legened to figure using bbox coordinates
    leg = xy.legend(bbox_to_anchor=bbox, loc='upper left', markerscale=2,ncol=1, borderaxespad=0., bbox_transform=fig.transFigure,
                   frameon=False,prop=legend_properties)
    


    plt.savefig('injury_profiles\\'+play+'_play_movements.png',bbox_inches='tight',dpi=96)
    plt.close()